<a href="https://colab.research.google.com/github/Ogunfool/Prognostics-Strategies-An-Aero-engine-Use-case/blob/main/Experimentations_To_Choose_Some_Residual_Similarity_Based_Model_Hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


 CONTENTS OF THIS NOTEBOOK.

*   This notebook contains 3-Metric Functions For RUL Prediction Error Evaluation for all validation set at 50,70 and 90% of the lifetime. The metrics used are:


1.   Root Mean Square Error (RMSE)
2.   MEA - Mean Absolute Error
1.   SCORE - The score metric for the PHM08 competition


*   It also contains LOCs to tune residual-similarity based model specific 
hyperparameters.
The Hyperparametrs tuned are:
1.   Choosing the best metric to measure similarity or degree of similarity between the validation data and train dataset. The MSE/RMSE, Cosine similarity, A combination of both and RMSE/cosine similarity degrees/rate (1/similarity value) are considered.
2.   Different nearest neighbor size (nearest) and L past timesteps considered i.e which training instances have the most similar degradation profiles to the test data in the past L steps?

1.   RUL Estimation based on nearest neighbours estimation method: Median of the RULs of the instances with most similar degradation profiles, Weighted average (weights = Ed=similarity degree = 1/similarity measure).






Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import math
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.decomposition import PCA

In [ ]:
# Visualize the histogram of the error for each breakpoint together with its probability distribution.
import seaborn as sns

Helper Functions

In [ ]:
np.set_printoptions(suppress=True, linewidth=100, precision=2)

In [ ]:
# Checkpoints - List and npy files
# np.save() - Saves a single array in a binary numpy format
def checkpoints(filename, checkpoint_data):
  np.save(filename, checkpoint_data)
  checkpoint_variable = np.load(filename + '.npy') #Load so that we always have an on-hand version of the checkpoint
  return(checkpoint_variable)

# List Checkpoint
def list_checkpoints(filename, checkpoint_data):
  np.save(filename, checkpoint_data, allow_pickle=True)
  checkpoint_variable = np.load(filename + '.npy', allow_pickle=True) #Load so that we always have an on-hand version of the checkpoint
  return(checkpoint_variable)

Load Data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving x-train.npy to x-train.npy
Saving x-val.npy to x-val.npy


In [ ]:
# Load HItrain and HIval back in
HItrain_df = pd.read_csv('/content/HItrain-df')
HIval_df = pd.read_csv('/content/HIval-df')

In [ ]:
# Load HItrain and HIval back in
temp_HItrain_df = pd.read_csv('/content/HItrain-df (3)')
temp_HIval_df = pd.read_csv('/content/HIval-df (3)')

Note: The dataframes HItrain and HIval dataframes have been updated with HI's from different models that have been constructed at the time of this experimentation (linear regression and Feedforward NN).

In [ ]:
apprx_HI_ANN = np.load('/content/HItrain_ANN.npy')
apprx_HI_ANN.shape

(12303,)

In [ ]:
HIval_df['apprx_HI_ANN'] = apprx_HI_ANN
HIval_df.head()

,Unit_id,Time,ori_HI,Maxcycle,RUL,apprx_HI,apprx_HI_ANN
0,201,1,1.000000,191,190,0.933014,0.898250
1,201,2,0.994737,191,189,0.763932,0.781614
2,201,3,0.989474,191,188,0.797754,0.796511
3,201,4,0.984211,191,187,0.739501,0.775219
4,201,5,0.978947,191,186,0.817956,0.820120


In [ ]:
apprx_HI_ANN = np.load('/content/HIval_ANN.npy')
apprx_HI_ANN.shape

(41456,)

In [ ]:
HItrain_df['apprx_HI_ANN'] = apprx_HI_ANN
HItrain_df.head()

,Unit_id,Time,ori_HI,Maxcycle,RUL,new_HI,apprx_HI,apprx_HI_ANN
0,1,1,1.000000,149,148,0.582822,0.546977,0.631525
1,1,2,0.993243,149,147,0.582912,0.581504,0.724908
2,1,3,0.986486,149,146,0.582965,0.601732,0.767940
3,1,4,0.979730,149,145,0.583290,0.726264,0.847148
4,1,5,0.972973,149,144,0.582879,0.568990,0.711482


In [ ]:
# Take median of the maxcycle of nearest residuals or neighbours
# Instances maxcycle_df
max_cycle_df = HItrain_df.groupby('Unit_id').max()['Time'].reset_index().rename(columns={'Time':'max_cycle'})
max_cycle_df.head()

,Unit_id,max_cycle
0,1,149
1,2,269
2,3,206
3,4,235
4,5,154


In [ ]:
max_cycle_df.set_index('Unit_id',inplace=True)
max_cycle_df.head()

,max_cycle
Unit_id,
1,149
2,269
3,206
4,235
5,154


# 3-Metric Functions for validation set at different percent lifetime.

Single Metric: This is a metric over all validation/Test set/UUt for LRP(Lifetime Record percentage) of 50,70 and 90%. In this work, UUts or N for validation set = 60.

In [ ]:
# Score Function
def score(errors):
  a1=10
  a2=13
  s1=0
  s2=0
  for err in errors:
    if err < 0:
      s1 += (np.exp(-1*(err/a1))) - 1
    if ((err > 0) or (err == 0)):
      s2 += (np.exp(err/a2)) - 1
  return [s1 , s2]

In [ ]:
def RUL_metrics(a,b,c):
  res_list = []
  RMSE = []
  MAE = []
  SCORE = []

  # Make an array for easy computation
  for res in [a,b,c]:
    res_list.append(np.array(res[:2]).T)
  res_array = np.array(res_list)

  for r in res_array:
    errors = r[0]-r[1]
    RMSE.append(mean_squared_error(r[0], r[1], squared=False))
    MAE.append(mean_absolute_error(r[0],r[1]))
    SCORE.append(score(errors))

  return RMSE, MAE, SCORE


In [ ]:
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

# Experiment 1: Experimenting the effect of changing L on different pct(s) and different HI approximation technique.

Experiment 1: Which instances have the most similar degradation profiles as the current validation data with LRP (50,70 or 90%) in the past L timesteps.

*   Try different L timesteps.
*   Note: Residual computed using RMSE.
*   Implementation detail: [Lt/5,Lt/5,Lt/3,Lt/2,Lt/1].
I refined the lenght function such that for different LRPs, there can be different lenghts (len).





Note: this code has been modified to accomodate changing L.

In [ ]:
# This is the residual function we are using because the lent can be larger than some instance's maxcycles.....
def residual_func(train_df, test_df, lent, name):
  lenf = math.ceil((test_df.shape[0]*0.5) / 5)
  no_instances = train_df['Unit_id'].unique().shape[0]
  res_mat = np.zeros((lent-lenf, no_instances))
  cur_val = test_df[name][lenf:lent]
  for id in train_df['Unit_id'].unique():
    # If it is already dead, then maybe you shouldn't be part of the computation OR add zeros below it
    temp_df = train_df[train_df['Unit_id']==id]
    if temp_df.shape[0] < lent:
      diff_arr = np.zeros((lent-lenf) - temp_df.shape[0])
      cur_train = temp_df[name][lenf:lent]
      cur_train = np.concatenate((cur_train, diff_arr), axis=0)
      res_mat[:,id-1] = cur_train
    else:
      temp_df = train_df[train_df['Unit_id']==id]
      cur_train = temp_df[name][lenf:lent]
      res_mat[:,id-1] = cur_train

  l = res_mat - cur_val.to_numpy().reshape((-1,1))
  residual = np.sqrt(np.mean(l**2, axis=0))
  return residual, res_mat, cur_val

In [ ]:
def neighbors(residual,nearest):
  n_neighbors = np.argsort(residual)[:nearest]
  return n_neighbors

Experiment 1a: HIs: Original HI, LR approximated HI and NN approximated HI - L = 5.

In [ ]:
ans, res_mat, cur_val = residual_func(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], lent=95, name='ori_HI')
neighbors(residual=ans,nearest=20)

array([ 23,  21, 194,  85, 173,  39, 153,  13,  20,  92,  58,  70, 138, 161, 185,  91,  89,  88,
        32,  25])

In [ ]:
ans, res_mat, cur_val = residual_func(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], lent=95, name='apprx_HI')
neighbors(residual=ans,nearest=20)

array([127,  49,  73, 133, 122, 146,  75,  14,  16,  98,  20,  43,  45,  11, 130, 113, 190,  60,
       174, 149])

In [ ]:
ans, res_mat, cur_val = residual_func(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], lent=95, name='apprx_HI_ANN')
neighbors(residual=ans,nearest=20)

array([ 14, 127,  73,  98,  49,  66, 146,  16,  47,  32,  75,  11, 170, 130,  12,  10,  63,  21,
       153, 172])

Experiment 1a: Changing Hi's: Original HI, LR approximated HI and NN approximated HI - L = 4

In [ ]:
ans, res_mat, cur_val = residual_func(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], lent=95, name='ori_HI')
neighbors(residual=ans,nearest=20)

array([ 23,  21, 194,  85, 173,  39, 153,  13,  20,  92,  58,  70, 138, 161, 185,  91,  89,  88,
        32,  25])

In [ ]:
ans, res_mat, cur_val = residual_func(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], lent=95, name='apprx_HI')
neighbors(residual=ans,nearest=20)

array([127,  49, 130,  98, 122,  75,  73, 133,  43,  16,  14,  45,  11,  20, 146, 149,  22, 113,
       190, 123])

In [ ]:
ans, res_mat, cur_val = residual_func(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], lent=95, name='apprx_HI_ANN')
neighbors(residual=ans,nearest=20)

array([127,  98,  14,  73,  49, 130,  16,  75,  66,  47,  32,  11, 146,  63,  12, 170,  22,  10,
        44,  36])

Experiment 1a: Changing Hi's: Original HI, LR approximated HI and NN approximated HI - L = 1.5

In [ ]:
ans, res_mat, cur_val = residual_func(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], lent=95, name='ori_HI')
neighbors(residual=ans,nearest=20)

array([ 23,  21, 194,  85, 173,  39, 153,  13,  20,  92,  58,  70, 138, 161, 185,  91,  89,  88,
        32,  25])

In [ ]:
ans, res_mat, cur_val = residual_func(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], lent=95, name='apprx_HI')
neighbors(residual=ans,nearest=20)

array([149,  49, 122,  98, 117, 133, 128, 113,  94,  38,  75, 127,  11, 130,  32,  12, 123,  36,
       156,  21])

In [ ]:
ans, res_mat, cur_val = residual_func(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], lent=95, name='apprx_HI_ANN')
neighbors(residual=ans,nearest=20)

array([ 66,  98,  49,  75,  11,  21,  38,  47, 156,  12,  73,  36, 153, 170, 127,  32,  63,  13,
       133,  44])

COMMENT: Observe the changes nearest neighbors chosen with chnages in L. It shows that the L hyperparameter needs to be tuned. Use error metrics to tune below.

In [ ]:
def lenght(test_df, pct):
 lent = math.ceil(test_df.shape[0]*pct)
 return lent

def neighbors(residual,nearest):
  n_neighbors = np.argsort(residual)[:nearest]
  return n_neighbors

In [ ]:
#  The function for new data will be a little different from that of val data
def RUL_estimator(train_df, test_df, max_cycle_df, pct, nearest, name):
  # Call Lenght Function
  lent = lenght(test_df,pct)

  # Call Residual Function
  residual = residual_func(train_df, test_df, lent, name)

  # Nearest neighbours function - where you can change the size of nearest neighbours
  n_neighbors = neighbors(residual,nearest)

  # RUL Estimation
  # We want the closest 50 that are not yet dead
  ens_RULs = max_cycle_df.loc[n_neighbors+1] - lent

  true_RUL = test_df.shape[0] - lent

  m = ens_RULs[ens_RULs>-10]
  m[m.isna()] = 0

  est_RUL = m['max_cycle'].median()

  return (est_RUL, true_RUL, (ens_RULs))
  # return lent, residual, n_neighbors

In [ ]:
# pct = 50%, lenf% = 1.5 
ans_1 = RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.5, nearest=50, name='ori_HI')

In [ ]:
# pct = 50%, lenf% = 2 
ans_2 = RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.5, nearest=50, name='ori_HI')

In [ ]:
# pct = 50%, lenf% = 3 
ans_3 = RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.5, nearest=50, name='ori_HI')

Put all your functions in one place.

In [ ]:
# This is the residual function we are using because the lent can be larger than some instance's maxcycles.....
def residual_func(train_df, test_df, lent, pct, name):
  lenf = math.ceil((test_df.shape[0]*pct) / 5)
  no_instances = train_df['Unit_id'].unique().shape[0]
  res_mat = np.zeros((lent-lenf, no_instances))
  cur_val = test_df[name][lenf:lent]
  for id in train_df['Unit_id'].unique():
    # If it is already dead, then maybe you shouldn't be part of the computation OR add zeros below it
    temp_df = train_df[train_df['Unit_id']==id]
    if temp_df.shape[0] < res_mat.shape[0]:
      diff_arr = np.zeros((lent-lenf) - temp_df.shape[0])
      cur_train = temp_df[name][lenf:]
      cur_train = np.concatenate((cur_train, diff_arr), axis=0)
      if cur_train.shape[0] < res_mat.shape[0]:
        tem = np.zeros((res_mat.shape[0]-cur_train.shape[0]))
        tem_arr = np.concatenate((cur_train, tem), axis=0)
        res_mat[:,id-1] = tem_arr
      else:
        res_mat[:,id-1] = cur_train
    else:
      temp_df = train_df[train_df['Unit_id']==id]
      cur_train = temp_df[name][lenf:lent]
      if cur_train.shape[0] < res_mat.shape[0]:
        tem = np.zeros((res_mat.shape[0]-cur_train.shape[0]))
        tem_arr = np.concatenate((cur_train, tem), axis=0)
        res_mat[:,id-1] = tem_arr
      else:
        res_mat[:,id-1] = cur_train

  l = res_mat - cur_val.to_numpy().reshape((-1,1))
  residual = np.sqrt(np.mean(l**2, axis=0))
  return residual

In [ ]:
def lenght(test_df, pct):
 lent = math.ceil(test_df.shape[0]*pct)
 return lent

def neighbors(residual,nearest):
  n_neighbors = np.argsort(residual)[:nearest]
  return n_neighbors

In [ ]:
#  The function for new data will be a little different from that of val data
def RUL_estimator(train_df, test_df, max_cycle_df, pct, nearest, name):
  # Call Lenght Function
  lent = lenght(test_df,pct)

  # Call Residual Function
  residual = residual_func(train_df, test_df, lent, pct, name)

  # Nearest neighbours function - where you can change the size of nearest neighbours
  n_neighbors = neighbors(residual,nearest)

  # RUL Estimation
  # We want the closest 50 that are not yet dead
  ens_RULs = max_cycle_df.loc[n_neighbors+1] - lent

  true_RUL = test_df.shape[0] - lent

  m = ens_RULs[ens_RULs>-10]
  m[m.isna()] = 0

  est_RUL = m['max_cycle'].median()

  return (est_RUL, true_RUL, (ens_RULs))
  # return lent, residual, n_neighbors

In [ ]:
def RUL_Estimator_coll(train_df, test_df, max_cycle_df, nearest, name):
  # Instantiate pct list:
  pct_list = [0.5,0.7,0.9]
  # Val_data unique ID
  inst_id = test_df['Unit_id'].unique()

  # Create a results dictionary:
  val_inst = {}
  for i in inst_id:
    val_inst[i] = {}
    for j in [0.5, 0.7, 0.9]:
      val_inst[i][j] = (0,0,())

  # Loop through all val_instances
  for k in inst_id:
    # Working with a single test set:
    for pct in [0.5, 0.7, 0.9]: 
      cur_val_df = test_df[test_df['Unit_id'] == k]
      val_inst[k][pct] = RUL_estimator(train_df=train_df, test_df=cur_val_df, max_cycle_df=max_cycle_df, pct=pct, nearest=nearest, name=name)

  return val_inst

In [ ]:
def result_analysis(val_inst):
  fif = []
  sev = []
  nine = []
  EST_RUL_5 = []
  TRUE_RUL_5 = []
  ENS_RUL_5 = []
  EST_RUL_7 = []
  TRUE_RUL_7 = []
  ENS_RUL_7 = []
  EST_RUL_9 = []
  TRUE_RUL_9 = []
  ENS_RUL_9 = []
  id_list = []
  # loop through all instances:
  for id,info in val_inst.items():
    id_list.append(id)

    EST_RUL_5.append(info[0.5][0])
    TRUE_RUL_5.append(info[0.5][1])
    ENS_RUL_5.append(info[0.5][2])

    EST_RUL_7.append(info[0.7][0])
    TRUE_RUL_7.append(info[0.7][1])
    ENS_RUL_7.append(info[0.5][2])

    EST_RUL_9.append(info[0.9][0])
    TRUE_RUL_9.append(info[0.9][1])
    ENS_RUL_9.append(info[0.5][2])

  fif = [EST_RUL_5, TRUE_RUL_5, ENS_RUL_5]
  sev = [EST_RUL_7, TRUE_RUL_7, ENS_RUL_7]
  nine = [EST_RUL_9, TRUE_RUL_9, ENS_RUL_9]
  
  return fif, sev, nine
  # Calculate RMSE

In [ ]:
def score(errors):
  a1=10
  a2=13
  s1=0
  s2=0
  for err in errors:
    if err < 0:
      s1 += (np.exp(-1*(err/a1))) - 1
    if ((err > 0) or (err == 0)):
      s2 += (np.exp(err/a2)) - 1
  return [s1 , s2]

In [ ]:
def RUL_metrics(a,b,c):
  res_list = []
  RMSE = []
  MAE = []
  SCORE = []

  # Make an array for easy computation
  for res in [a,b,c]:
    res_list.append(np.array(res[:2]).T)
  res_array = np.array(res_list)

  for r in res_array:
    errors = r[0]-r[1]
    RMSE.append(mean_squared_error(r[0], r[1], squared=False))
    MAE.append(mean_absolute_error(r[0],r[1]))
    SCORE.append(score(errors))

  return RMSE, MAE, SCORE


Experiment 1c: Find the L that gives the lowest metrics for pct (50,70 and 90). Call all above functions and changing L at each call.

In [ ]:
# L = 1.5
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='ori_HI')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[3.2596012026013246, 2.2638462845343543, 0.3535533905932738]

In [ ]:
MAE

[3.25, 2.25, 0.25]

In [ ]:
SCORE

[[0.7689263561692603, 0], [0.5054281748479112, 0], [0.05127109637602412, 0.0]]

In [ ]:
# L = 1.5
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[21.319005605327842, 14.212670403551895, 4.596194077712559]

In [ ]:
MAE

[16.5, 11.0, 3.25]

In [ ]:
SCORE

[[0.3498588075760032, 9.05120278750086],
 [0.22140275816016985, 3.657419495658906],
 [0, 0.6487212707001282]]

In [ ]:
# L = 2
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[21.319005605327842, 12.806248474865697, 7.424621202458749]

In [ ]:
MAE

[16.5, 10.0, 5.25]

In [ ]:
SCORE

[[0.3498588075760032, 9.05120278750086],
 [0.22140275816016985, 2.993289728752191],
 [0, 1.2427264876594117]]

In [ ]:
# L = 3
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[17.804493814764857, 12.103718436910205, 6.363961030678928]

In [ ]:
MAE

[14.0, 9.5, 4.5]

In [ ]:
SCORE

[[0.3498588075760032, 5.841978355514408],
 [0.22140275816016985, 2.6976308645666607],
 [0, 0.9983217280388539]]

In [ ]:
# L = 4
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[20.615528128088304, 12.103718436910205, 6.7175144212722016]

In [ ]:
MAE

[16.0, 9.5, 4.75]

In [ ]:
SCORE

[[0.3498588075760032, 8.307022574766952],
 [0.22140275816016985, 2.6976308645666607],
 [0, 1.0766774376128017]]

In [ ]:
# L = 5
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[20.615528128088304, 10.350120772242226, 6.010407640085654]

In [ ]:
MAE

[16.0, 8.25, 4.25]

In [ ]:
SCORE

[[0.3498588075760032, 8.307022574766952],
 [0.22140275816016985, 2.0507413156095278],
 [0, 0.9229224801241105]]

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[17.804493814764857, 7.211102550927978, 4.242640687119285]

In [ ]:
MAE

[14.0, 6.0, 3.0]

In [ ]:
SCORE

[[0.3498588075760032, 5.841978355514408],
 [0.22140275816016985, 1.1581055339484458],
 [0, 0.5865128974999683]]

Comments.


1.   First, recall that we said using the original health index (manually formulated for this problem) gives the best result; which shows the impact or effect of HI estimation method when using a similarity-based method.
It also tells us that to use this HI estimation method, we need a very high r-score value.
From this experiment, we could see that with changing L and using the original_HI, the effect is close to nothing or very minimal while the effect is more obvious when we use approximate HI's. So if we decide to use approx_HI or PCA method, this is a parameter that should be tuned.
2.   Since the approximate values are most affected by change in L, the experiment was carried out with the apprx_HI_ANN and from this experiment, the best L is 3 for 50%, 5 for 70% and 90%. However, using the full lenght is the best so far with MAE of [14,6,3].



Load Data - Add PCA results to dataframe

In [ ]:
temp_HItrain_df.head()

,Unit_id,Time,ori_HI,Maxcycle,RUL,new_HI,apprx_HI,HI_PCA,new_HI_PCA
0,1,1,1.000000,149,148,0.582822,0.546977,-0.859024,-1.192820
1,1,2,0.993243,149,147,0.582912,0.581504,-0.628524,-1.197747
2,1,3,0.986486,149,146,0.582965,0.601732,-0.404679,-1.202504
3,1,4,0.979730,149,145,0.583290,0.726264,-1.859147,-1.171129
4,1,5,0.972973,149,144,0.582879,0.568990,-0.310075,-1.204507


In [ ]:
HItrain_df[['HI_PCA','new_HI_PCA']] = temp_HItrain_df[['HI_PCA','new_HI_PCA']]
HItrain_df.head()

,Unit_id,Time,ori_HI,Maxcycle,RUL,new_HI,apprx_HI,apprx_HI_ANN,HI_PCA,new_HI_PCA
0,1,1,1.000000,149,148,0.582822,0.546977,0.631525,-0.859024,-1.192820
1,1,2,0.993243,149,147,0.582912,0.581504,0.724908,-0.628524,-1.197747
2,1,3,0.986486,149,146,0.582965,0.601732,0.767940,-0.404679,-1.202504
3,1,4,0.979730,149,145,0.583290,0.726264,0.847148,-1.859147,-1.171129
4,1,5,0.972973,149,144,0.582879,0.568990,0.711482,-0.310075,-1.204507


In [ ]:
HIval_df[['HI_PCA','new_HI_PCA']] = temp_HIval_df[['HI_PCA','new_HI_PCA']]
HIval_df.head()

,Unit_id,Time,ori_HI,Maxcycle,RUL,apprx_HI,apprx_HI_ANN,HI_PCA,new_HI_PCA
0,201,1,1.000000,191,190,0.933014,0.898250,-3.938672,-1.124376
1,201,2,0.994737,191,189,0.763932,0.781614,-2.332807,-1.160676
2,201,3,0.989474,191,188,0.797754,0.796511,-2.512478,-1.156681
3,201,4,0.984211,191,187,0.739501,0.775219,-2.180602,-1.164047
4,201,5,0.978947,191,186,0.817956,0.820120,-3.277371,-1.139486


Do experiment 1c with PCA and compare results to that of apprx_HI_ANN.

In [ ]:
# L = 1.5
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[21.670832932769336, 19.49679460834524, 10.253048327204938]

In [ ]:
MAE

[16.75, 14.75, 7.25]

In [ ]:
SCORE

[[0.3498588075760032, 9.445318067055567],
 [0.22140275816016985, 7.29277468155062],
 [0, 2.0507413156095278]]

In [ ]:
# L = 2
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[19.560802642018555, 19.49679460834524, 9.192388155425117]

In [ ]:
MAE

[15.25, 14.75, 6.5]

In [ ]:
SCORE

[[0.3498588075760032, 7.29277468155062],
 [0.22140275816016985, 7.29277468155062],
 [0, 1.718281828459045]]

In [ ]:
# L = 3
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[23.07866980568854, 19.849433241279208, 8.838834764831844]

In [ ]:
MAE

[17.75, 15.0, 6.25]

In [ ]:
SCORE

[[0.3498588075760032, 11.182493960703473],
 [0.22140275816016985, 7.61794066227959],
 [0, 1.6157175603482896]]

In [ ]:
# L = 4
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[22.02271554554524, 16.32482771731451, 8.838834764831844]

In [ ]:
MAE

[17.0, 12.5, 6.25]

In [ ]:
SCORE

[[0.3498588075760032, 9.854886905439232],
 [0.22140275816016985, 4.866339056828965],
 [0, 1.6157175603482896]]

In [ ]:
# L = 5
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[22.02271554554524, 15.620499351813308, 9.192388155425117]

In [ ]:
MAE

[17.0, 12.0, 6.5]

In [ ]:
SCORE

[[0.3498588075760032, 9.854886905439232],
 [0.22140275816016985, 4.432001640742895],
 [0, 1.718281828459045]]

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[18.85802216564611, 14.564511663629508, 10.253048327204938]

In [ ]:
MAE

[14.75, 11.25, 7.25]

In [ ]:
SCORE

[[0.3498588075760032, 6.678786589066299],
 [0.22140275816016985, 3.840040444150536],
 [0, 2.0507413156095278]]

Comments.
1.   Just incase you decide to use PCA, L = 1.5 gave better results for 50%, L = 5 gave better result for 70%, L = 4 for 90%. L = 4 could be a good point for all pcts.

1.   This shows that the L is a hyperparameter that should be tuned for any HI construction model you decide to use. Once we are able to get a HI approximation model that estimates the HI the best from the sensor values, we will tune the L again.

 






# Experiment 2: Choosing the similarity metric.

Similarity Metric: The metric we use to select the train examples that have the closest degradation profile to the test data. 

*   The tested metrics are RMSE, MSE, Cosine similarity, degrees of similarity (1/similarity measure).

*   And method in this paper: Liu, Yingchao, Xiaofeng Hu, and Wenjuan Zhang. "Remaining useful life prediction based on health index similarity." Reliability Engineering & System Safety 185 (2019): 502-510.



In [ ]:
def lenght(test_df, pct):
 lent = math.ceil(test_df.shape[0]*pct)
 return lent

# when using similarity degree, use argsort(- residual)
def neighbors(residual,nearest):
  n_neighbors = np.argsort(-residual)[:nearest]
  return n_neighbors

In [ ]:
#  The function for new data will be a little different from that of val data
def RUL_estimator(train_df, test_df, max_cycle_df, pct, nearest, name):
  # Call Lenght Function
  lent = lenght(test_df,pct)

  # Call Residual Function
  residual = residual_func(train_df, test_df, lent, name)

  # Nearest neighbours function - where you can change the size of nearest neighbours
  n_neighbors = neighbors(residual,nearest)

  # RUL Estimation
  # We want the closest 50 that are not yet dead
  ens_RULs = max_cycle_df.loc[n_neighbors+1] - lent

  true_RUL = test_df.shape[0] - lent

  m = ens_RULs[ens_RULs>-10]
  m[m.isna()] = 0

  est_RUL = m['max_cycle'].median()

  return (est_RUL, true_RUL, (ens_RULs))
  # return lent, residual, n_neighbors

In [ ]:
def RUL_Estimator_coll(train_df, test_df, max_cycle_df, nearest, name):
  # Instantiate pct list:
  pct_list = [0.5,0.7,0.9]
  # Val_data unique ID
  inst_id = test_df['Unit_id'].unique()

  # Create a results dictionary:
  val_inst = {}
  for i in inst_id:
    val_inst[i] = {}
    for j in [0.5, 0.7, 0.9]:
      val_inst[i][j] = (0,0,())

  # Loop through all val_instances
  for k in inst_id:
    # Working with a single test set:
    for pct in [0.5, 0.7, 0.9]: 
      cur_val_df = test_df[test_df['Unit_id'] == k]
      val_inst[k][pct] = RUL_estimator(train_df=train_df, test_df=cur_val_df, max_cycle_df=max_cycle_df, pct=pct, nearest=nearest, name=name)

  return val_inst


In [ ]:
def result_analysis(val_inst):
  fif = []
  sev = []
  nine = []
  EST_RUL_5 = []
  TRUE_RUL_5 = []
  ENS_RUL_5 = []
  EST_RUL_7 = []
  TRUE_RUL_7 = []
  ENS_RUL_7 = []
  EST_RUL_9 = []
  TRUE_RUL_9 = []
  ENS_RUL_9 = []
  id_list = []
  # loop through all instances:
  for id,info in val_inst.items():
    id_list.append(id)

    EST_RUL_5.append(info[0.5][0])
    TRUE_RUL_5.append(info[0.5][1])
    ENS_RUL_5.append(info[0.5][2])

    EST_RUL_7.append(info[0.7][0])
    TRUE_RUL_7.append(info[0.7][1])
    ENS_RUL_7.append(info[0.5][2])

    EST_RUL_9.append(info[0.9][0])
    TRUE_RUL_9.append(info[0.9][1])
    ENS_RUL_9.append(info[0.5][2])

  fif = [EST_RUL_5, TRUE_RUL_5, ENS_RUL_5]
  sev = [EST_RUL_7, TRUE_RUL_7, ENS_RUL_7]
  nine = [EST_RUL_9, TRUE_RUL_9, ENS_RUL_9]
  
  return fif, sev, nine
  # Calculate RMSE



In [ ]:
def score(errors):
  a1=10
  a2=13
  s1=0
  s2=0
  for err in errors:
    if err < 0:
      s1 += (np.exp(-1*(err/a1))) - 1
    if ((err > 0) or (err == 0)):
      s2 += (np.exp(err/a2)) - 1
  return [s1 , s2]

In [ ]:
def RUL_metrics(a,b,c):
  res_list = []
  RMSE = []
  MAE = []
  SCORE = []

  # Make an array for easy computation
  for res in [a,b,c]:
    res_list.append(np.array(res[:2]).T)
  res_array = np.array(res_list)

  for r in res_array:
    errors = r[0]-r[1]
    RMSE.append(mean_squared_error(r[0], r[1], squared=False))
    MAE.append(mean_absolute_error(r[0],r[1]))
    SCORE.append(score(errors))

  return RMSE, MAE, SCORE


In [ ]:
from numpy.linalg import norm

In [ ]:
# This is the residual function we are using because the lent can be larger than some instance's maxcycles.....
def residual_func(train_df, test_df, lent, name):
  no_instances = train_df['Unit_id'].unique().shape[0]
  res_mat = np.zeros((lent, no_instances))
  cur_val = test_df[name][:lent]
  for id in train_df['Unit_id'].unique():
    # If it is already dead, then maybe you shouldn't be part of the computation OR add zeros below it
    temp_df = train_df[train_df['Unit_id']==id]
    if temp_df.shape[0] < lent:
      diff_arr = np.zeros(lent- temp_df.shape[0])
      cur_train = temp_df[name][:lent]
      cur_train = np.concatenate((cur_train, diff_arr), axis=0)
      res_mat[:,id-1] = cur_train
    else:
      temp_df = train_df[train_df['Unit_id']==id]
      cur_train = temp_df[name][:lent]
      res_mat[:,id-1] = cur_train

  # Cos similarity
  residuals_1 = []
  for rr in res_mat.T:
    A = rr
    B = cur_val
    residual_1 = np.dot(A,B)/(norm(A)*norm(B))
    residuals_1.append(residual_1)
  residual_cos = 1/np.array(residuals_1)  

  # RMSE
  l = res_mat - cur_val.to_numpy().reshape((-1,1))
  residual_2 = 1/(np.sqrt(np.mean(l**2, axis=0)))

  # Sum residuals
  residual = residual_cos + residual_2

  return residual

MSE Metric

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[17.804493814764857, 7.211102550927978, 4.242640687119285]

In [ ]:
MAE

[14.0, 6.0, 3.0]

In [ ]:
SCORE

[[0.3498588075760032, 5.841978355514408],
 [0.22140275816016985, 1.1581055339484458],
 [0, 0.5865128974999683]]

RMSE Metric

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[17.804493814764857, 7.211102550927978, 4.242640687119285]

In [ ]:
MAE

[14.0, 6.0, 3.0]

In [ ]:
SCORE

[[0.3498588075760032, 5.841978355514408],
 [0.22140275816016985, 1.1581055339484458],
 [0, 0.5865128974999683]]

MAE Metric

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[17.804493814764857, 11.40175425099138, 5.303300858899107]

In [ ]:
MAE

[14.0, 9.0, 3.75]

In [ ]:
SCORE

[[0.3498588075760032, 5.841978355514408],
 [0.22140275816016985, 2.4238622637752645],
 [0, 0.7805513738412788]]

MAE degree/rate Metric

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[17.804493814764857, 11.40175425099138, 5.303300858899107]

In [ ]:
MAE

[14.0, 9.0, 3.75]

In [ ]:
SCORE

[[0.3498588075760032, 5.841978355514408],
 [0.22140275816016985, 2.4238622637752645],
 [0, 0.7805513738412788]]

RMSE and MSE degree/rate Metric

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[17.804493814764857, 7.211102550927978, 4.242640687119285]

In [ ]:
MAE

[14.0, 6.0, 3.0]

In [ ]:
SCORE

[[0.3498588075760032, 5.841978355514408],
 [0.22140275816016985, 1.1581055339484458],
 [0, 0.5865128974999683]]

Cos Similarity

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[17.804493814764857, 7.211102550927978, 1.0606601717798212]

In [ ]:
MAE

[14.0, 6.0, 0.75]

In [ ]:
SCORE

[[0.3498588075760032, 5.841978355514408],
 [0.22140275816016985, 1.1581055339484458],
 [0.16183424272828306, 0.0]]

Cos Similarity and RMSE + Similarity Degrees.

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI_ANN')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[17.804493814764857, 7.211102550927978, 4.242640687119285]

In [ ]:
MAE

[14.0, 6.0, 3.0]

In [ ]:
SCORE

[[0.3498588075760032, 5.841978355514408],
 [0.22140275816016985, 1.1581055339484458],
 [0, 0.5865128974999683]]

Comment: MSE and RMSE returns the same results; so does MSE/RMSE degrees.

*   MAE does not perform as good as MSE/RMSE; as expected MAE and MAE degree gives the same result.

*   Cosine similarity - This gave a better result at pct = 50%.

*   RMSE + Cos + Similarity degree (paper 1): This does not do any better than the RMSE/MSE metric. Not even as good as cos similarity stand alone.

*   Conclusion: Use RMSE or Cos similarity.






# Weighted RUL Experiment

RUL = w(i) * RUL(i); w(i) = percentage (%) of similarity = 1/RMSE.
We want the most similar neighbours to have more effect on the RUL estimation rather than all neighbors having the same weight.

*   Note: The best neighbour size has been tuned in the "A_Residual_Similarity_Based_Method_for_Remaining_Useful_Life_Prediction_Using_Degradation_Degree" notebook, and 50 neighbours works well.



NOTE: The residual_func and RUL_estimator functions have been modified to show this effect.

In [ ]:
# This is the residual function we are using because the lent can be larger than some instance's maxcycles.....
def residual_func(train_df, test_df, lent, name):
  no_instances = train_df['Unit_id'].unique().shape[0]
  res_mat = np.zeros((lent, no_instances))
  cur_val = test_df[name][:lent]
  for id in train_df['Unit_id'].unique():
    # If it is already dead, then maybe you shouldn't be part of the computation OR add zeros below it
    temp_df = train_df[train_df['Unit_id']==id]
    if temp_df.shape[0] < lent:
      diff_arr = np.zeros(lent- temp_df.shape[0])
      cur_train = temp_df[name][:lent]
      cur_train = np.concatenate((cur_train, diff_arr), axis=0)
      res_mat[:,id-1] = cur_train
    else:
      temp_df = train_df[train_df['Unit_id']==id]
      cur_train = temp_df[name][:lent]
      res_mat[:,id-1] = cur_train

  l = res_mat - cur_val.to_numpy().reshape((-1,1))
  residual = np.sqrt(np.mean(l**2, axis=0))
  res_deg = 1/(residual+1000)
  return residual, res_deg

In [ ]:
def lenght(test_df, pct):
 lent = math.ceil(test_df.shape[0]*pct)
 return lent

def neighbors(residual,nearest):
  n_neighbors = np.argsort(residual)[:nearest]
  return n_neighbors

In [ ]:
#  The function for new data will be a little different from that of val data
def RUL_estimator(train_df, test_df, max_cycle_df, pct, nearest, name):
  # Call Lenght Function
  lent = lenght(test_df,pct)

  # Call Residual Function
  residual, res_deg = residual_func(train_df, test_df, lent, name)

  # Nearest neighbours function - where you can change the size of nearest neighbours
  n_neighbors = neighbors(residual,nearest)

  # RUL Estimation
  # We want the closest 50 that are not yet dead
  ens_RULs = max_cycle_df.loc[n_neighbors+1] - lent

  true_RUL = test_df.shape[0] - lent

  m = ens_RULs[ens_RULs>-10]
  m[m.isna()] = 0

  est_RUL = np.sum((np.multiply(m['max_cycle'], res_deg[:nearest])))

  return (est_RUL, true_RUL, (ens_RULs))
  # return lent, residual, n_neighbors

In [ ]:
RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.7, nearest=50, name='ori_HI')

In [ ]:
def RUL_Estimator_coll(train_df, test_df, max_cycle_df, nearest, name):
  # Instantiate pct list:
  pct_list = [0.5,0.7,0.9]
  # Val_data unique ID
  inst_id = test_df['Unit_id'].unique()

  # Create a results dictionary:
  val_inst = {}
  for i in inst_id:
    val_inst[i] = {}
    for j in [0.5, 0.7, 0.9]:
      val_inst[i][j] = (0,0,())

  # Loop through all val_instances
  for k in inst_id:
    # Working with a single test set:
    for pct in [0.5, 0.7, 0.9]: 
      cur_val_df = test_df[test_df['Unit_id'] == k]
      val_inst[k][pct] = RUL_estimator(train_df=train_df, test_df=cur_val_df, max_cycle_df=max_cycle_df, pct=pct, nearest=nearest, name=name)

  return val_inst


In [ ]:
def result_analysis(val_inst):
  fif = []
  sev = []
  nine = []
  EST_RUL_5 = []
  TRUE_RUL_5 = []
  ENS_RUL_5 = []
  EST_RUL_7 = []
  TRUE_RUL_7 = []
  ENS_RUL_7 = []
  EST_RUL_9 = []
  TRUE_RUL_9 = []
  ENS_RUL_9 = []
  id_list = []
  # loop through all instances:
  for id,info in val_inst.items():
    id_list.append(id)

    EST_RUL_5.append(info[0.5][0])
    TRUE_RUL_5.append(info[0.5][1])
    ENS_RUL_5.append(info[0.5][2])

    EST_RUL_7.append(info[0.7][0])
    TRUE_RUL_7.append(info[0.7][1])
    ENS_RUL_7.append(info[0.5][2])

    EST_RUL_9.append(info[0.9][0])
    TRUE_RUL_9.append(info[0.9][1])
    ENS_RUL_9.append(info[0.5][2])

  fif = [EST_RUL_5, TRUE_RUL_5, ENS_RUL_5]
  sev = [EST_RUL_7, TRUE_RUL_7, ENS_RUL_7]
  nine = [EST_RUL_9, TRUE_RUL_9, ENS_RUL_9]
  
  return fif, sev, nine
  # Calculate RMSE



In [ ]:
def score(errors):
  a1=10
  a2=13
  s1=0
  s2=0
  for err in errors:
    if err < 0:
      s1 += (np.exp(-1*(err/a1))) - 1
    if ((err > 0) or (err == 0)):
      s2 += (np.exp(err/a2)) - 1
  return [s1 , s2]

In [ ]:
def RUL_metrics(a,b,c):
  res_list = []
  RMSE = []
  MAE = []
  SCORE = []

  # Make an array for easy computation
  for res in [a,b,c]:
    res_list.append(np.array(res[:2]).T)
  res_array = np.array(res_list)

  for r in res_array:
    errors = r[0]-r[1]
    RMSE.append(mean_squared_error(r[0], r[1], squared=False))
    MAE.append(mean_absolute_error(r[0],r[1]))
    SCORE.append(score(errors))

  return RMSE, MAE, SCORE


HI Constructed with PCA 

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[43.606170140911864, 24.80266727204535, 7.664859454481117]

In [ ]:
MAE

[32.29771155750175, 18.50960170603246, 5.419874097105419]

In [ ]:
SCORE

[[0.3498588075760032, 113.21806480385763],
 [0.22140275816016985, 13.787539165435343],
 [0, 1.3021115895541668]]

Original HI

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='ori_HI')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[2.1213212940395016, 1.414214160825573, 0.0001130875457130223]

In [ ]:
MAE

[1.5014199551798706, 1.000919967276443, 7.996497044142174e-05]

In [ ]:
SCORE

[[0.3501428389412491, 0], [0.2215867685432924, 0], [0, 1.2302378818551674e-05]]

Comment: The weighted estimate just added more error to the estimation for PCA but does a good job for original HI.
NOTE: This section needs to be modified.....

Original HI + L (FULL) + Metric (RMSE) + Est_RUL (MEDIAN)

In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='ori_HI')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
RMSE

[3.2596012026013246, 2.2638462845343543, 0.3535533905932738]

In [ ]:
MAE

[3.25, 2.25, 0.25]

In [ ]:
SCORE

[[0.7689263561692603, 0], [0.5054281748479112, 0], [0.05127109637602412, 0.0]]

Just manually examining what our chosen model is doing.


*   f = estimated RUL
*   f1 = True RUL



In [ ]:
(f,f1,f2) = RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.5, nearest=50, name='ori_HI')
print(f,f1)

97.0 95


In [ ]:
(f,f1,f2) = RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.7, nearest=50, name='ori_HI')
print(f,f1)

59.0 57


In [ ]:
(f,f1,f2) = RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.9, nearest=50, name='ori_HI')
print(f,f1)

21.0 19


For approximate HI we can see that the bound is getting tighter as we get more data. 

In [ ]:
(f,f1,f2) = RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.5, nearest=50, name='apprx_HI_ANN')
print(f,f1)

114.5 95


In [ ]:
(f,f1,f2) = RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.7, nearest=50, name='apprx_HI_ANN')
print(f,f1)

61.5 57


In [ ]:
(f,f1,f2) = RUL_estimator(train_df=HItrain_df, test_df=HIval_df[HIval_df['Unit_id'] == 201], max_cycle_df=max_cycle_df, pct=0.9, nearest=50, name='apprx_HI_ANN')
print(f,f1)

16.5 19


# Actual RUL Estimation Functions with no experiments.

See metrics on validation dataset using the best hyperparameters tuned above for every model.

In [ ]:
# This is the residual function we are using because the lent can be larger than some instance's maxcycles.....
def residual_func(train_df, test_df, lent, name):
  no_instances = train_df['Unit_id'].unique().shape[0]
  res_mat = np.zeros((lent, no_instances))
  cur_val = test_df[name][:lent]
  for id in train_df['Unit_id'].unique():
    # If it is already dead, then maybe you shouldn't be part of the computation OR add zeros below it
    temp_df = train_df[train_df['Unit_id']==id]
    if temp_df.shape[0] < lent:
      diff_arr = np.zeros(lent- temp_df.shape[0])
      cur_train = temp_df[name][:lent]
      cur_train = np.concatenate((cur_train, diff_arr), axis=0)
      res_mat[:,id-1] = cur_train
    else:
      temp_df = train_df[train_df['Unit_id']==id]
      cur_train = temp_df[name][:lent]
      res_mat[:,id-1] = cur_train

  l = res_mat - cur_val.to_numpy().reshape((-1,1))
  residual = np.sqrt(np.mean(l**2, axis=0))
  return residual

In [ ]:
def lenght(test_df, pct):
 lent = math.ceil(test_df.shape[0]*pct)
 return lent

def neighbors(residual,nearest):
  n_neighbors = np.argsort(residual)[:nearest]
  return n_neighbors

In [ ]:
#  The function for new data will be a little different from that of val data
def RUL_estimator(train_df, test_df, max_cycle_df, pct, nearest, name):
  # Call Lenght Function
  lent = lenght(test_df,pct)

  # Call Residual Function
  residual = residual_func(train_df, test_df, lent, name)

  # Nearest neighbours function - where you can change the size of nearest neighbours
  n_neighbors = neighbors(residual,nearest)

  # RUL Estimation
  # We want the closest 50 that are not yet dead
  ens_RULs = max_cycle_df.loc[n_neighbors+1] - lent

  true_RUL = test_df.shape[0] - lent

  m = ens_RULs[ens_RULs>-10]
  m[m.isna()] = 0

  est_RUL = m['max_cycle'].median()

  return (est_RUL, true_RUL, (ens_RULs))
  # return lent, residual, n_neighbors

In [ ]:
def RUL_Estimator_coll(train_df, test_df, max_cycle_df, nearest, name):
  # Instantiate pct list:
  pct_list = [0.5,0.7,0.9]
  # Val_data unique ID
  inst_id = test_df['Unit_id'].unique()

  # Create a results dictionary:
  val_inst = {}
  for i in inst_id:
    val_inst[i] = {}
    for j in [0.5, 0.7, 0.9]:
      val_inst[i][j] = (0,0,())

  # Loop through all val_instances
  for k in inst_id:
    # Working with a single test set:
    for pct in [0.5, 0.7, 0.9]: 
      cur_val_df = test_df[test_df['Unit_id'] == k]
      val_inst[k][pct] = RUL_estimator(train_df=train_df, test_df=cur_val_df, max_cycle_df=max_cycle_df, pct=pct, nearest=nearest, name=name)

  return val_inst


In [ ]:
def result_analysis(val_inst):
  fif = []
  sev = []
  nine = []
  EST_RUL_5 = []
  TRUE_RUL_5 = []
  ENS_RUL_5 = []
  EST_RUL_7 = []
  TRUE_RUL_7 = []
  ENS_RUL_7 = []
  EST_RUL_9 = []
  TRUE_RUL_9 = []
  ENS_RUL_9 = []
  id_list = []
  # loop through all instances:
  for id,info in val_inst.items():
    id_list.append(id)

    EST_RUL_5.append(info[0.5][0])
    TRUE_RUL_5.append(info[0.5][1])
    ENS_RUL_5.append(info[0.5][2])

    EST_RUL_7.append(info[0.7][0])
    TRUE_RUL_7.append(info[0.7][1])
    ENS_RUL_7.append(info[0.5][2])

    EST_RUL_9.append(info[0.9][0])
    TRUE_RUL_9.append(info[0.9][1])
    ENS_RUL_9.append(info[0.5][2])

  fif = [EST_RUL_5, TRUE_RUL_5, ENS_RUL_5]
  sev = [EST_RUL_7, TRUE_RUL_7, ENS_RUL_7]
  nine = [EST_RUL_9, TRUE_RUL_9, ENS_RUL_9]
  
  return fif, sev, nine
  # Calculate RMSE



In [ ]:
def score(errors):
  a1=10
  a2=13
  s1=0
  s2=0
  for err in errors:
    if err < 0:
      s1 += (np.exp(-1*(err/a1))) - 1
    if ((err > 0) or (err == 0)):
      s2 += (np.exp(err/a2)) - 1
  return [s1 , s2]

In [ ]:
def RUL_metrics(a,b,c):
  res_list = []
  RMSE = []
  MAE = []
  SCORE = []

  # Make an array for easy computation
  for res in [a,b,c]:
    res_list.append(np.array(res[:2]).T)
  res_array = np.array(res_list)

  for r in res_array:
    errors = r[0]-r[1]
    RMSE.append(mean_squared_error(r[0], r[1], squared=False))
    MAE.append(mean_absolute_error(r[0],r[1]))
    SCORE.append(score(errors))

  return RMSE, MAE, SCORE


This is an updated Health Index dataframe: See the "Health Index Prediction Model"  and "Unsupervised Learning Health Index Construction Models" to see how these health indices were constructed or estimated. Below experiment shows the RUL estimation scores at 50,70 and 90% of the lifetime of the validation dataset.

In [ ]:
HItrain_df.head()

,Unit_id,Time,ori_HI,Maxcycle,RUL,new_HI,apprx_HI,HI_PCA,new_HI_PCA,HI_PCA_L,HI_PCA_Lz
0,1,1,1.000000,149,148,0.582822,0.546977,-0.859024,-1.192820,-0.794293,0.187668
1,1,2,0.993243,149,147,0.582912,0.581504,-0.628524,-1.197747,-0.701790,0.197499
2,1,3,0.986486,149,146,0.582965,0.601732,-0.404679,-1.202504,-0.405560,0.228983
3,1,4,0.979730,149,145,0.583290,0.726264,-1.859147,-1.171129,-1.376195,0.125823
4,1,5,0.972973,149,144,0.582879,0.568990,-0.310075,-1.204507,-0.534039,0.215328


In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='ori_HI')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
print(RMSE,MAE,SCORE)

[3.2596012026013246, 2.2638462845343543, 0.3535533905932738] [3.25, 2.25, 0.25] [[0.7689263561692603, 0], [0.5054281748479112, 0], [0.05127109637602412, 0.0]]


In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='apprx_HI')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
print(RMSE,MAE,SCORE)

[12.55487952948972, 15.972632844963288, 11.667261889578034] [10.25, 12.25, 8.25] [[0.3498588075760032, 2.8426177733663716], [0.22140275816016985, 4.644994542233771], [0, 2.5581144982364594]]


In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
print(RMSE,MAE,SCORE)

[18.85802216564611, 14.564511663629508, 10.253048327204938] [14.75, 11.25, 7.25] [[0.3498588075760032, 6.678786589066299], [0.22140275816016985, 3.840040444150536], [0, 2.0507413156095278]]


In [ ]:
# L is Full, k_components = 3
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA_L')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
print(RMSE,MAE,SCORE)

[20.967236346261757, 12.103718436910205, 3.5355339059327378] [16.25, 9.5, 2.5] [[0.3498588075760032, 8.671957984132842], [0.22140275816016985, 2.6976308645666607], [0, 0.46904919384901667]]


In [ ]:
# L is Full, k_components = 8
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA_L')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
print(RMSE,MAE,SCORE)

[20.967236346261757, 14.212670403551895, 3.5355339059327378] [16.25, 11.0, 2.5] [[0.3498588075760032, 8.671957984132842], [0.22140275816016985, 3.657419495658906], [0, 0.46904919384901667]]


In [ ]:
# L is Full, k_components = 5
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA_L')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
print(RMSE,MAE,SCORE)

[20.967236346261757, 14.212670403551895, 3.5355339059327378] [16.25, 11.0, 2.5] [[0.3498588075760032, 8.671957984132842], [0.22140275816016985, 3.657419495658906], [0, 0.46904919384901667]]


In [ ]:
# L is Full, k_components = 2
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA_L')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
print(RMSE,MAE,SCORE)

[20.967236346261757, 14.212670403551895, 3.8890872965260113] [16.25, 11.0, 2.75] [[0.3498588075760032, 8.671957984132842], [0.22140275816016985, 3.657419495658906], [0, 0.5266517261980206]]


In [ ]:
# L is Full
val_inst = RUL_Estimator_coll(train_df=HItrain_df, test_df=HIval_df, max_cycle_df=max_cycle_df, nearest=50, name='HI_PCA_Lz')
a,b,c = result_analysis(val_inst)
RMSE, MAE, SCORE = RUL_metrics(a,b,c)

In [ ]:
print(RMSE,MAE,SCORE)

[23.430749027719962, 19.144189719076646, 9.192388155425117] [18.0, 14.5, 6.5] [[0.3498588075760032, 11.660178782560491], [0.22140275816016985, 6.979877631319886], [0, 1.718281828459045]]
